In [1]:
import sys
print(sys.executable)

/home/user/ml_env/bin/python


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7MGcGjNozVjlwdHeLsSy")
project = rf.workspace("cophe").project("sperm-motion-detection")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [3]:
!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=300 imgsz=640 plots=True

Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7925MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/user/ml_env/Sperm-Motion-Detection-1/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

In [16]:
!yolo task=detect mode=val \
    model=/home/user/ml_env/runs/detect/Train/weights/best.pt \
    data={dataset.location}/data.yaml \
    imgsz=640 \
    batch=16 \
    conf=0.25 \
    iou=0.6 \
    plots=True \
    save_txt \
    save_conf \
    name=Validate


73082.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7925MiB)
YOLO11s summary (fused): 238 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs
val: Scanning /home/user/ml_env/Sperm-Motion-Detection-1/valid/labels.cache... 5
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all       5834     130500      0.997      0.991      0.994      0.952
                     0       5781     121768      0.994       0.99      0.995      0.928
                     1       1991       4327          1          1      0.995      0.995
                     2       2665       4405      0.996      0.983      0.992      0.933
Speed: 0.2ms preprocess, 4.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/validation_results
💡 Learn more at https://docs.ultralytics.com/modes/val


In [17]:
!yolo task=detect mode=predict \
    model=/home/user/ml_env/runs/detect/Train/weights/best.pt \
    conf=0.25 \
    source={dataset.location}/test/images \
    save=True \
    save_txt=True \
    save_conf=True \
    project=/home/user/ml_env/runs/detect \
    name=Test \
    exist_ok=True

73440.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7925MiB)
YOLO11s summary (fused): 238 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs

image 1/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1014_jpg.rf.8cff2e466e2d17c2ed62369d73315d43.jpg: 640x640 36 0s, 2 1s, 3.4ms
image 2/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1015_jpg.rf.1a27c8324a3153d56ed97b61200941fd.jpg: 640x640 36 0s, 2 1s, 3.3ms
image 3/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1039_jpg.rf.19215e7fa6fa44f7b2dcb04657cf7146.jpg: 640x640 36 0s, 2 1s, 3.3ms
image 4/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_103_jpg.rf.d196da1046f95effe6abde2e4b4f67b8.jpg: 640x640 40 0s, 3.3ms
image 5/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1072_jpg.rf.eca5c732c627d68b5ffc0559d3306216.jpg: 640x640 36 0s, 2 1s, 3.3ms
image 6/2939 /home/user/ml_env/Sperm-Motion-Detection

In [35]:
import os
import shutil

# Source directory where model results are saved
source_dir = '/home/user/ml_env/runs/detect/'

# Destination directory for organized datasets
dest_dir = '/media/user/data/Mano/Mano-Project/Project-Mano/Output'

# List of dataset numbers
datasets = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47, 52, 54, 60, 82]

for dataset in datasets:
    dataset_str = f"{dataset:02d}"
    dataset_dir = os.path.join(dest_dir, dataset_str)
    
    # Create dataset directories
    os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels_ftid'), exist_ok=True)

    frame_list = []

    # Process files
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.startswith(f"{dataset_str}_frame_"):
                src_path = os.path.join(root, file)
                if file.endswith('.jpg'):
                    dst_path = os.path.join(dataset_dir, 'images', file)
                    shutil.copy2(src_path, dst_path)
                    frame_list.append(dst_path)
                elif file.endswith('.txt') and not file.endswith('_with_ftid.txt'):
                    dst_path = os.path.join(dataset_dir, 'labels', file)
                    shutil.copy2(src_path, dst_path)
                elif file.endswith('_with_ftid.txt'):
                    new_name = file.replace('_with_ftid.txt', '.txt')
                    dst_path = os.path.join(dataset_dir, 'labels_ftid', new_name)
                    shutil.copy2(src_path, dst_path)

    # Create dataset.txt file
    with open(os.path.join(dataset_dir, f'{dataset_str}.txt'), 'w') as f:
        for frame_path in sorted(frame_list):
            f.write(f'{frame_path}\n')

    print(f'Organized dataset {dataset_str}')

print('All datasets organized successfully.')


Organized dataset 11
Organized dataset 12
Organized dataset 13
Organized dataset 14
Organized dataset 15
Organized dataset 19
Organized dataset 21
Organized dataset 22
Organized dataset 23
Organized dataset 24
Organized dataset 29
Organized dataset 30
Organized dataset 35
Organized dataset 36
Organized dataset 38
Organized dataset 47
Organized dataset 52
Organized dataset 54
Organized dataset 60
Organized dataset 82
All datasets organized successfully.


In [38]:
#Trial1
import os
import shutil

# Source directories
source_dir = '/home/user/ml_env/runs/detect/'
train_dir = '/home/user/ml_env/Sperm-Motion-Detection-1/train'
val_dir = '/home/user/ml_env/Sperm-Motion-Detection-1/valid'

# Destination directory for organized datasets
dest_dir = '/media/user/data/Mano/Mano-Project/Project-Mano/Output'

# List of dataset numbers
datasets = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47, 52, 54, 60, 82]

def find_image(image_name, directories):
    for directory in directories:
        image_path = os.path.join(directory, image_name)
        if os.path.exists(image_path):
            return image_path
    return None

for dataset in datasets:
    dataset_str = f"{dataset:02d}"
    dataset_dir = os.path.join(dest_dir, dataset_str)
    
    # Create dataset directories
    os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels_ftid'), exist_ok=True)

    frame_list = []

    # Process files
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.startswith(f"{dataset_str}_frame_"):
                src_path = os.path.join(root, file)
                if file.endswith('.jpg'):
                    dst_path = os.path.join(dataset_dir, 'images', file)
                    shutil.copy2(src_path, dst_path)
                    frame_list.append(dst_path)
                elif file.endswith('.txt') and not file.endswith('_with_ftid.txt'):
                    dst_path = os.path.join(dataset_dir, 'labels', file)
                    shutil.copy2(src_path, dst_path)
                elif file.endswith('_with_ftid.txt'):
                    new_name = file.replace('_with_ftid.txt', '.txt')
                    dst_path = os.path.join(dataset_dir, 'labels_ftid', new_name)
                    shutil.copy2(src_path, dst_path)

    # Check for missing images in train and valid directories
    for label_file in os.listdir(os.path.join(dataset_dir, 'labels')):
        image_name = label_file.replace('.txt', '.jpg')
        if image_name not in os.listdir(os.path.join(dataset_dir, 'images')):
            image_path = find_image(image_name, [train_dir, val_dir])
            if image_path:
                dst_path = os.path.join(dataset_dir, 'images', image_name)
                shutil.copy2(image_path, dst_path)
                frame_list.append(dst_path)
                print(f"Retrieved missing image: {image_name}")
            else:
                print(f"Warning: Image not found for {image_name}")

    # Create dataset.txt file
    with open(os.path.join(dataset_dir, f'{dataset_str}.txt'), 'w') as f:
        for frame_path in sorted(frame_list):
            f.write(f'{frame_path}\n')

    print(f'Organized dataset {dataset_str}')

print('All datasets organized successfully.')

Organized dataset 11
Organized dataset 12
Organized dataset 13
Organized dataset 14
Organized dataset 15
Organized dataset 19
Organized dataset 21
Organized dataset 22
Organized dataset 23
Organized dataset 24
Organized dataset 29
Organized dataset 30
Organized dataset 35
Organized dataset 36
Organized dataset 38
Organized dataset 47
Organized dataset 52
Organized dataset 54
Organized dataset 60
Organized dataset 82
All datasets organized successfully.


In [39]:
#trial2
import os
import shutil
import glob

# Source directories
source_dir = '/home/user/ml_env/runs/detect/'
train_dir = '/home/user/ml_env/Sperm-Motion-Detection-1/train'
val_dir = '/home/user/ml_env/Sperm-Motion-Detection-1/valid'

# Destination directory for organized datasets
dest_dir = '/media/user/data/Mano/Mano-Project/Project-Mano/Output2'

# List of dataset numbers
datasets = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47, 52, 54, 60, 82]

def find_image(image_name, directories):
    for directory in directories:
        # Try exact match
        image_path = os.path.join(directory, image_name)
        if os.path.exists(image_path):
            return image_path
        
        # Try finding the image with a similar name
        pattern = f"{image_name.split('_jpg')[0]}*.jpg"
        matches = glob.glob(os.path.join(directory, pattern))
        if matches:
            return matches[0]
    return None

for dataset in datasets:
    dataset_str = f"{dataset:02d}"
    dataset_dir = os.path.join(dest_dir, dataset_str)
    
    # Create dataset directories
    os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels_ftid'), exist_ok=True)

    frame_list = []
    missing_images = []

    # Process files
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.startswith(f"{dataset_str}_frame_"):
                src_path = os.path.join(root, file)
                if file.endswith('.jpg'):
                    dst_path = os.path.join(dataset_dir, 'images', file)
                    shutil.copy2(src_path, dst_path)
                    frame_list.append(dst_path)
                elif file.endswith('.txt') and not file.endswith('_with_ftid.txt'):
                    dst_path = os.path.join(dataset_dir, 'labels', file)
                    shutil.copy2(src_path, dst_path)
                elif file.endswith('_with_ftid.txt'):
                    new_name = file.replace('_with_ftid.txt', '.txt')
                    dst_path = os.path.join(dataset_dir, 'labels_ftid', new_name)
                    shutil.copy2(src_path, dst_path)

    # Check for missing images in train and valid directories
    for label_file in os.listdir(os.path.join(dataset_dir, 'labels')):
        image_name = label_file.replace('.txt', '.jpg')
        if image_name not in os.listdir(os.path.join(dataset_dir, 'images')):
            image_path = find_image(image_name, [train_dir, val_dir])
            if image_path:
                dst_path = os.path.join(dataset_dir, 'images', os.path.basename(image_path))
                shutil.copy2(image_path, dst_path)
                frame_list.append(dst_path)
                print(f"Retrieved missing image: {image_name}")
            else:
                missing_images.append(image_name)
                print(f"Warning: Image not found for {image_name}")

    # Create dataset.txt file
    with open(os.path.join(dataset_dir, f'{dataset_str}.txt'), 'w') as f:
        for frame_path in sorted(frame_list):
            f.write(f'{frame_path}\n')

    # Create missing_images.txt file
    if missing_images:
        with open(os.path.join(dataset_dir, 'missing_images.txt'), 'w') as f:
            for image_name in missing_images:
                f.write(f'{image_name}\n')

    print(f'Organized dataset {dataset_str}')
    print(f'Total images: {len(frame_list)}')
    print(f'Missing images: {len(missing_images)}')

print('All datasets organized successfully.')


Organized dataset 11
Total images: 128
Missing images: 328
Organized dataset 12
Total images: 163
Missing images: 278
Organized dataset 13
Total images: 146
Missing images: 293
Organized dataset 14
Total images: 162
Missing images: 275
Organized dataset 15
Total images: 141
Missing images: 290
Organized dataset 19
Total images: 146
Missing images: 305
Organized dataset 21
Total images: 149
Missing images: 285
Organized dataset 22
Total images: 150
Missing images: 291
Organized dataset 23
Total images: 129
Missing images: 263
Organized dataset 24
Total images: 147
Missing images: 282
Organized dataset 29
Total images: 146
Missing images: 303
Organized dataset 30
Total images: 143
Missing images: 298
Organized dataset 35
Total images: 163
Missing images: 287
Organized dataset 36
Total images: 136
Missing images: 279
Organized dataset 38
Total images: 149
Missing images: 311
Organized dataset 47
Total images: 155
Missing images: 279
Organized dataset 52
Total images: 138
Missing images: 2

In [8]:
from ultralytics import YOLO
import os
import glob
import shutil

# Load your trained model
model = YOLO('/home/user/ml_env/runs/detect/Train/weights/best.pt')

# Define dataset numbers
datasets = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47, 52, 54, 60, 82]

# Define directories
dirs = {
    'test': '/home/user/ml_env/Sperm-Motion-Detection-1/test/images',
    'train': '/home/user/ml_env/Sperm-Motion-Detection-1/train/images',
    'valid': '/home/user/ml_env/Sperm-Motion-Detection-1/valid/images'
}

for dataset in datasets:
    dataset_str = f"{dataset:02d}"
    print(f"\nProcessing dataset {dataset_str}")
    
    # Collect all images for this dataset from all directories
    dataset_images = []
    for dir_name, dir_path in dirs.items():
        if os.path.exists(dir_path):
            images = glob.glob(os.path.join(dir_path, f"{dataset_str}_frame_*.jpg"))
            if images:
                dataset_images.extend(images)
                print(f"Found {len(images)} images in {dir_name}")
    
    if dataset_images:
        # Sort images to ensure frame order
        dataset_images.sort()
        
        # Create output directory
        output_dir = f'/home/user/ml_env/runs/detect/track/dataset_{dataset_str}'
        os.makedirs(output_dir, exist_ok=True)
        
        try:
            results = model.track(
                source=dataset_images,
                tracker="bytetrack",
                conf=0.3,
                iou=0.5,
                show=False,
                save=True,
                project=output_dir,
                name='tracks'
            )
            print(f"Successfully tracked {len(dataset_images)} frames for dataset {dataset_str}")
        except Exception as e:
            print(f"Error processing dataset {dataset_str}: {str(e)}")
    else:
        print(f"No images found for dataset {dataset_str}")



Processing dataset 11
Found 128 images in test
Found 1014 images in train
Found 328 images in valid


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Error processing dataset 11: 'bytetrack' does not exist

Processing dataset 12
Found 163 images in test
Found 1029 images in train
Found 278 images in valid


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ul

In [14]:
from ultralytics import YOLO
import os
import glob
import shutil

# Load your trained model
model = YOLO('/home/user/ml_env/runs/detect/Train/weights/best.pt')

# Define dataset numbers
datasets = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47, 52, 54, 60, 82]

# Define directories
dirs = {
    'test': '/home/user/ml_env/Sperm-Motion-Detection-1/test/images',
    'train': '/home/user/ml_env/Sperm-Motion-Detection-1/train/images',
    'valid': '/home/user/ml_env/Sperm-Motion-Detection-1/valid/images'
}

for dataset in datasets:
    dataset_str = f"{dataset:02d}"
    print(f"\nProcessing dataset {dataset_str}")
    
    # Collect all images for this dataset from all directories
    dataset_images = []
    for dir_name, dir_path in dirs.items():
        if os.path.exists(dir_path):
            images = glob.glob(os.path.join(dir_path, f"{dataset_str}_frame_*.jpg"))
            if images:
                dataset_images.extend(images)
                print(f"Found {len(images)} images in {dir_name}")
    
    if dataset_images:
        # Sort images to ensure frame order
        dataset_images.sort()
        
        # Create output directory
        output_dir = f'/home/user/ml_env/runs/detect/track/dataset_{dataset_str}'
        os.makedirs(output_dir, exist_ok=True)
        
        try:
            # Process images in batches to manage memory
            for i in range(0, len(dataset_images), 100):  # Process 100 images at a time
                batch_images = dataset_images[i:i+100]
                results = model.track(
                    source=batch_images,
                    tracker="botsort.yaml",  # Changed from bytetrack to botsort.yaml
                    conf=0.3,
                    iou=0.5,
                    show=False,
                    save=True,
                    project=output_dir,
                    name=f'tracks_batch_{i//100}',
                    stream=True  # Added to prevent RAM accumulation
                )
                
                # Process results as they stream
                for r in results:
                    boxes = r.boxes  # Process tracking results
                
            print(f"Successfully tracked {len(dataset_images)} frames for dataset {dataset_str}")
        except Exception as e:
            print(f"Error processing dataset {dataset_str}: {str(e)}")
    else:
        print(f"No images found for dataset {dataset_str}")



Processing dataset 11
Found 128 images in test
Found 1014 images in train
Found 328 images in valid

0: 640x640 36 0s, 2 1s, 4.6ms
1: 640x640 36 0s, 2 1s, 4.6ms
2: 640x640 36 0s, 2 1s, 4.6ms
3: 640x640 39 0s, 4.6ms
4: 640x640 36 0s, 2 1s, 4.6ms
5: 640x640 37 0s, 2 1s, 4.6ms
6: 640x640 36 0s, 2 1s, 4.6ms
7: 640x640 36 0s, 2 1s, 4.6ms
8: 640x640 36 0s, 2 1s, 4.6ms
9: 640x640 37 0s, 2 1s, 4.6ms
10: 640x640 36 0s, 2 1s, 4.6ms
11: 640x640 36 0s, 2 1s, 4.6ms
12: 640x640 36 0s, 2 1s, 4.6ms
13: 640x640 40 0s, 2 1s, 4.6ms
14: 640x640 36 0s, 2 1s, 4.6ms
15: 640x640 36 0s, 2 1s, 4.6ms
16: 640x640 40 0s, 2 1s, 4.6ms
17: 640x640 36 0s, 2 1s, 4.6ms
18: 640x640 36 0s, 2 1s, 4.6ms
19: 640x640 36 0s, 2 1s, 4.6ms
20: 640x640 36 0s, 2 1s, 4.6ms
21: 640x640 36 0s, 2 1s, 4.6ms
22: 640x640 35 0s, 2 1s, 4.6ms
23: 640x640 35 0s, 2 1s, 4.6ms
24: 640x640 36 0s, 2 1s, 4.6ms
25: 640x640 34 0s, 2 1s, 4.6ms
26: 640x640 34 0s, 2 1s, 4.6ms
27: 640x640 34 0s, 2 1s, 4.6ms
28: 640x640 34 0s, 2 1s, 4.6ms
29: 640x640 34 

In [15]:
import cv2
import os
import glob

# Input and output paths
input_dir = '/home/user/ml_env/runs/detect/track/dataset_11/tracks'
output_dir = '/home/user/ml_env/runs/detect/track/dataset_11'
output_video = os.path.join(output_dir, 'tracked_video.mp4')

# Get all images in the directory
images = glob.glob(os.path.join(input_dir, '*.jpg'))
images.sort()  # Sort to maintain frame order

if images:
    # Read first image to get dimensions
    frame = cv2.imread(images[0])
    height, width, _ = frame.shape
    
    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (width, height))
    
    # Process each image
    for image_path in images:
        frame = cv2.imread(image_path)
        out.write(frame)
    
    # Release video writer
    out.release()
    print(f"Video saved to: {output_video}")
else:
    print("No images found in the input directory")


Video saved to: /home/user/ml_env/runs/detect/track/dataset_11/tracked_video.mp4


In [5]:
from ultralytics import YOLO

# Load an official or custom YOLOv8 model
model = YOLO('/home/user/ml_env/yolov8x.pt')  # Ensure the model path is correct

# Perform tracking with the model
results = model.track(
    source="/media/user/data/Mano/Mano-Project/Project-Mano/VISEM-Tracking/VISEM_Tracking_Train_v4/Train/12/12.mp4",
    show=False,          # Set to True if you want to display the tracking live
    save=True,           # Save the output video
    project="test_result",  # Output directory for results
    name="tracked_output",  # Subdirectory name
    persist=True         # Ensures consistent object IDs across frames
)

print("Tracking completed. Results saved in 'test_result/tracked_output'.")




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1470) /media/user/data/Mano/Mano-Project/Project-Mano/VISEM-Tracking/VISEM_Tracking_Train_v4/Train/12/12.mp4: 480x640 (no detections), 27.5ms
video 1/1 (frame 2/1470) /media/user/data/Mano/Mano-Project/Project-Mano/VISEM-Tracking/VISEM_Tracking_Train_v4/Train/12/12.mp4: 480x640 (no detections), 20.5ms
video 1/1 (frame 3/1470) /media/user/data/Mano/Mano-Project/Project-Mano/VISEM-Tracking/VISEM_Tracking_Train_v4/Train/12/12.mp4: 480x6